In [1]:
# import packages
import pandas as pd
import numpy as np 
import re 
import time
import json

# Reddit

In [8]:
import praw
import requests
import pickle

pickle_in = open("individual_info","rb")
individual_info = pickle.load(pickle_in)

In [13]:
reddit = praw.Reddit(username=individual_info['username'],
                      password=individual_info["password"],
                     client_id=individual_info['client_id'], 
                     client_secret=individual_info['client_secret'], 
                     user_agent=individual_info['user_agent'],
                    )

In [25]:
def extract_data(topic):
    subreddit=reddit.subreddit(topic)
    hot_python = subreddit.hot(limit=1000)
    title_reply=[]
    for submission in hot_python:
        if not submission.stickied:
            comments = submission.comments
            for comment in comments:
                try:
                    title_reply.append({"body":comment.body,
                                        "reply":[reply.body for reply in comment.replies]})                 
                except:
                    pass
                
                
                
    questions=[]
    answers=[]
    for i in range(len(title_reply)):
        questions.append(list(title_reply)[i]['body'])
        answers.append(list(title_reply)[i]["reply"]) 
        
    return questions,answers

In [29]:
# extract data for those topics in the topics1 list 
topics1=["datascience","Python","pythoncoding",'python_netsec',"pythontips","pythonprojects2"]
Questions=[]
Answers=[]
for topic in topics1:
    q,a=extract_data(topic)
    Questions.append(q)
    Answers.append(a)

In [58]:
topics2=['Numpy','statsmodels',"matplotlib","datacleansing","anaconda","datavisualization"]
for topic in topics2:
    q,a=extract_data(topic)
    Questions.append(q)
    Answers.append(a)

In [62]:
len(Questions),len(Answers)

(13, 13)

In [63]:
Q=[]
A=[]
for i in range(len(Questions)):
    for j in range(len(Questions[i])):
        Q.append(Questions[i][j])
        A.append(Answers[i][j])

In [104]:
len(Q),len(A)

(11284, 11284)

In [120]:
AS=[]
for i in A:
    try:
        AS.append(i[0])
    except:
        AS.append(None)

In [122]:
len(AS)

11284

In [96]:
list_of_answers=[]
for i in A:
    for j in i:
        list_of_answers.append(j)

In [108]:
len(list_of_answers)

6654

In [114]:
def rearrange_date(list_of_answers):
    new_questions = []
    new_answers = []
    for i in range(len(list_of_answers)-1):
        new_questions.append(list_of_answers[i])
        new_answers.append(list_of_answers[i+1])
    return new_questions,new_answers


In [115]:
new_questions,new_answers=rearrange_date(list_of_answers) # create new questions and answers 

In [116]:
len(new_questions),len(new_answers) # double check the length

(6653, 6653)

In [117]:
new_questions[0],new_answers[0]

("This. My organization is still new to it as well and so it's not very strict. For us, often a DS and a MLE will often team up to convert the DS's poor excuse for a pipeline into something that will actually work in production. But it's usually after the DS has developed, pitched, and been approved to move forward with whatever they were working on.",
 'My org is very similar but the ML and Eng teams are also separate. Also the ML people are called "data scientists" and the DS people are called "decision scientists" so it\'s not at all confusing.')

In [133]:
questions=Q+new_questions
answers=AS+new_answers

In [134]:
len(questions),len(answers)

(17937, 17937)

In [130]:
pickle_out1=open('questions', 'wb')
pickle_out2=open('answers', 'wb')

pickle.dump(reddit, pickle_out1)
pickle.dump(reddit, pickle_out2)

pickle_out1.close()
pickle_out2.close()

In [131]:
questions = pickle.load(open("questions","rb"))
answers = pickle.load(open("answers","rb"))

In [137]:
df=pd.DataFrame()
df['question']=questions
df["answer"]=answers

In [138]:
df.head()

,question,answer
0,At my company - data scientists are in the Pro...,This. My organization is still new to it as we...
1,"At my company, the ML Engineers focus on build...",It's the rare DS that can move to MLE in my op...
2,"At my company, we have data scientists, ML eng...",None
3,"At my last job, the data scientists were basic...",None
4,They seem similar at first but they're in two ...,None


In [141]:
df.to_csv("df_python_reddit.csv")